In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.base import BaseEstimator
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import os
import pickle
import torch.optim as optim
from tqdm import tqdm
from IPython.display import clear_output
import time
import math
import openpyxl
import matplotlib.font_manager as fm 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
class Tre:
    def __init__(self):
        self.root = {}

    def insert(self, sequence):
        node = self.root
        for item in sequence:
            if item not in node:
                node[item] = {}
            node = node[item]
        node['end'] = True

    def search(self, sequence):
        node = self.root
        for item in sequence:
            if item not in node:
                return False
            node = node[item]
        return 'end' in node

    def extract_features(self, sequence):
        features = []
        for i in range(len(sequence)):
            for j in range(i+1, len(sequence)+1):
                subsequence = sequence[i:j]
                if self.search(subsequence):
                    features.append(subsequence)
                else:
                    break
        return features

class SelfAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(SelfAttention, self).__init__()
        self.hidden_dim = hidden_dim
        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x):
        query = self.query(x)
        key = self.key(x)
        value = self.value(x)

        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.hidden_dim)
        attention_weights = nn.functional.softmax(attention_scores, dim=-1)
        attended_values = torch.matmul(attention_weights, value)

        return attended_values

class WellLoggingDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, num_layers, output_dim):
        super(LSTMRegressor, self).__init__()

        self.lstm1 = nn.LSTM(input_dim, hidden_dim1, 1, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(hidden_dim1 * 2, hidden_dim2, 1, batch_first=True, bidirectional=True)
        self.lstm3 = nn.LSTM(hidden_dim2 * 2, hidden_dim3, 1, batch_first=True, bidirectional=True)

        self.fc = nn.Linear(hidden_dim3 * 2, output_dim)

    def forward(self, src):
        src = src.unsqueeze(1)

        lstm_output1, _ = self.lstm1(src)
        lstm_output2, _ = self.lstm2(lstm_output1)
        lstm_output3, _ = self.lstm3(lstm_output2)

        output = self.fc(lstm_output3[:, -1, :])

        return output

def train_model(model, train_loader, val_loader, epochs, learning_rate, patience):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    best_model_params = None
    epochs_without_improvement = 0

    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        batch_losses = []
        for batch_idx, (features, labels) in enumerate(train_loader):
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            batch_losses.append(loss.item())

        epoch_loss = sum(batch_losses) / len(batch_losses)
        train_losses.append(epoch_loss)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_params = model.state_dict()
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            break

    model.load_state_dict(best_model_params)
    return model, train_losses, val_losses

def test_model(model, test_loader, learning_rate, num_iterations):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()
    for iteration in range(num_iterations):
        for batch_idx, (features, labels) in enumerate(test_loader):
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    test_predictions = []
    with torch.no_grad():
        for features, _ in test_loader:
            features = features.to(device)
            outputs = model(features)
            test_predictions.extend(outputs.cpu().numpy())

    return np.array(test_predictions)

def preprocess_data(features, target, test_size=0.3, random_state=42):
    feature_scaler = RobustScaler()
    target_scaler = RobustScaler()

    features_scaled = feature_scaler.fit_transform(features)
    target_scaled = target_scaler.fit_transform(target.reshape(-1, 1))

    features_train, features_test, target_train, target_test = train_test_split(
        features_scaled, target_scaled, test_size=test_size, random_state=random_state
    )

    return features_train, features_test, target_train, target_test, feature_scaler, target_scaler

def extract_tre_features(data, max_length):
    tre = Tre()
    for sequence in data:
        tre.insert(sequence)

    tre_features = []
    for sequence in data:
        features = tre.extract_features(sequence)
        feature_vector = {}
        for subsequence in features:
            if len(subsequence) <= max_length:
                feature_vector[tuple(subsequence)] = 1
        tre_features.append(feature_vector)

    num_samples = len(tre_features)
    num_features = max(len(feature_dict) for feature_dict in tre_features)
    ttt_features_array = np.zeros((num_samples, num_features), dtype=int)
    for i, feature_dict in enumerate(tre_features):
        for j, (subsequence, value) in enumerate(feature_dict.items()):
            ttt_features_array[i, j] = value

    return ttt_features_array


In [ ]:
for step in range(5, 6):
    misses = ['DEPTH', 'GR', 'RS', 'RD', 'DEN', 'DTC']
    feature_names = misses[:step] + misses[step+1:]
    targ = misses[step]
    print('calculate_missing:', targ)

    file_path = r'/Recon_Model-main/data/end_data.csv'
    training_data = pd.read_csv(file_path)

    features = training_data[feature_names].values
    target = training_data[targ].values.reshape(-1, 1)
    layer_feature = training_data['Layer'] 

    onehot_encoder = OneHotEncoder(handle_unknown='ignore')

    feature_scaler = RobustScaler()
    target_scaler = RobustScaler()
    features = np.nan_to_num(features)
    target = np.nan_to_num(target)

    # Hot coding layer column
    layer_onehot = onehot_encoder.fit_transform(layer_feature.values.reshape(-1, 1)).toarray()
    features_encoded = np.concatenate((features, layer_onehot), axis=1)

    # normalization
    features_scaled = feature_scaler.fit_transform(features_encoded)
    target_scaled = target_scaler.fit_transform(target)

    # Split data
    features_train, features_test, target_train, target_test = train_test_split(features_scaled, target_scaled, test_size=0.3, random_state=42)

    # Extract features
    max_length = 5
    tre_features_train = extract_tre_features(features_train, max_length)
    tre_features_test = extract_tre_features(features_test, max_length)

    # Combine features with original features
    features_train_combined = np.concatenate((features_train, tre_features_train), axis=1)
    features_test_combined = np.concatenate((features_test, tre_features_test), axis=1)

    train_dataset = WellLoggingDataset(features_train_combined, target_train)
    val_dataset = WellLoggingDataset(features_test_combined, target_test)
    test_dataset = WellLoggingDataset(features_test_combined, target_test)

    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

    input_dim = features_train_combined.shape[1]
    hidden_dim1 = 256
    hidden_dim2 = 512
    hidden_dim3 = 256
    num_layers = 5
    output_dim = 1
    epochs = 90
    learning_rate = 0.001
    patience = 10

    model = LSTMRegressor(input_dim, hidden_dim1, hidden_dim2, hidden_dim3, num_layers, output_dim)
    model, train_losses, val_losses = train_model(model, train_loader, val_loader, epochs, learning_rate, patience)

    # TTT:  Dynamically update model parameters in the test phase
    test_learning_rate = 0.001
    num_iterations = 5
    test_predictions = test_model(model, test_loader, test_learning_rate, num_iterations)

    test_predictions = test_predictions.reshape(-1, 1)
    target_test = target_test.reshape(-1, 1)

    # Inverse standardization
    target_test_original = target_scaler.inverse_transform(target_test)
    test_predictions_original = target_scaler.inverse_transform(test_predictions)

    # Calculation index
    test_rmse_original_scale = np.sqrt(mean_squared_error(target_test_original, test_predictions_original))
    test_mae_original_scale = mean_absolute_error(target_test_original, test_predictions_original)
    test_r2_original_scale = r2_score(target_test_original, test_predictions_original)
    
    
    # Save the trained model
    save_dir = os.path.abspath('/Recon_Model-main/model') 
   
    model_filename = os.path.join(save_dir, f'model_{targ}.pkl')
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)
    print(f"Model for {targ} saved to {model_filename}")

    # Save the thermal encoder
    onehot_filename = os.path.join(save_dir, f'onehot_encoder_{targ}.pkl')
    with open(onehot_filename, 'wb') as f:  
        pickle.dump(onehot_encoder, f)
    print(f"OneHotEncoder saved to {onehot_filename}")

    # Save feature scaler
    feature_scaler_filename = os.path.join(save_dir, f'feature_scaler_{targ}.pkl')
    with open(feature_scaler_filename, 'wb') as f:
        pickle.dump(feature_scaler, f)
    print(f"Feature scaler saved to {feature_scaler_filename}")

    # Save target variable scaler
    target_scaler_filename = os.path.join(save_dir, f'target_scaler_{targ}.pkl')
    with open(target_scaler_filename, 'wb') as f:
        pickle.dump(target_scaler, f)  
    print(f"Target scaler for {targ} saved to {target_scaler_filename}")

